In [39]:
from algs_lib import *


In [40]:
def clip_to_threshold(vec, c):
    curr_norm = np.linalg.norm(vec)
    if curr_norm <= c:
        return vec
    clip_ratio = c / curr_norm
    return [vec[i]*clip_ratio for i in range(len(vec))]

In [41]:
def add_noise(scale):
    return np.random.laplace(0, scale)
# global sensitivity is C/n i think?
# so scale should be (C/n) / \epsilon per elem?

In [42]:
def calc_posterior(mi, prior=0.5, prec = 100000):
    test_vals = [x / prec for x in range(1, prec)]
    max_t = None
    for t in test_vals:
        if t*np.log(t/prior)+(1-t)*np.log((1-t)/(1-prior)) <= mi:
            if  max_t is None or t > max_t:
                max_t = t
    return max_t

def dp_epsilon_to_posterior_success(epsilon):
    return 1 - 1./(1+np.exp(epsilon))

def dp_ps_to_epsilon(ps):
    return np.log(ps / (1-ps))

In [43]:
calc_posterior(1.)

0.99999

In [45]:
def hybrid_noise_auto(train_x, train_y, subsample_rate, num_classes,
    eta, regularize=None, num_trees=None, tree_depth = None, max_mi = 0.5, num_dims = None):

    sec_v = max_mi / 2
    sec_beta = max_mi - sec_v
    r = calc_r(train_x)
    gamma = 0.01
    avg_dist = 0.
    curr_est = None
    converged = False
    curr_trial = 0

    if num_classes is None:
        num_classes = len(set(train_y))

    assert subsample_rate >= num_classes

    est_y = {}
    prev_ests = None
    # 10*c*v
    seed = np.random.randint(1, 100000)

    s1 = None # only relevant for PCA
    
    while not converged:
        shuffled_x, shuffled_y = shuffle(train_x, train_y)
        
        shuffled_x, shuffled_y = get_samples_safe(shuffled_x, shuffled_y, num_classes, subsample_rate)
        
        output = np.average(shuffled_x, axis=0)

        for ind in range(len(output)):
            if ind not in est_y:
                est_y[ind] = []
            est_y[ind].append(output[ind])

        if curr_trial % 10 == 0:        
            if prev_ests is None:
                prev_ests = {}
                for ind in est_y:
                    prev_ests[ind] = np.var(est_y[ind])
            else:
                converged = True
                for ind in est_y:
                    if abs(np.var(est_y[ind]) - prev_ests[ind]) > eta:
                        converged = False
                if not converged:
                    for ind in est_y:
                        prev_ests[ind] = np.var(est_y[ind])
        curr_trial += 1
    fin_var = {ind: np.var(est_y[ind]) for ind in est_y}

    noise = {}
    sqrt_total_var = sum([fin_var[x]**0.5 for x in fin_var])
    for ind in fin_var:
        noise[ind] = 1./(2*max_mi) * fin_var[ind]**0.5 * sqrt_total_var
    return noise

In [71]:
train_x, train_y, test_x, test_y, num_classes, train_len = gen_iris(normalize=False)

norms = [np.linalg.norm(x) for x in train_x]
print(max(norms))

10.5256828757093


In [72]:
true_mean = np.average(train_x, axis=0)

mi_range = [1.0, 0.25, 0.0625, 0.015625]
posterior_success_rates = [calc_posterior(mi) for mi in mi_range]
epsilon_vals = [dp_ps_to_epsilon(ps) for ps in posterior_success_rates]

print(epsilon_vals)
print([x for x in posterior_success_rates])

[11.51291546492478, 1.6426117097961406, 0.7304317044395013, 0.3563228120191924]
[0.99999, 0.83789, 0.6749, 0.58815]


In [73]:
# DP MEAN
dp_dists = {}
num_trials = 1000
for eps in epsilon_vals:
    avg_dist_dp = {}
    for i in range(1, 110):
        clip_budget = i / 10
        clipped_train_x = [clip_to_threshold(train_x[i], clip_budget) for i in range(len(train_x))]
        released_mean = np.average(clipped_train_x, axis=0)
        clip_dist = np.linalg.norm(released_mean - true_mean)
        dist = 0.
        sensitivity = clip_budget / train_len
        for _ in range(num_trials):
            released_mean = np.average(clipped_train_x, axis=0)
            for ind in range(len(released_mean)):
                sensitivity = clip_budget / train_len
                released_mean[ind] += add_noise(sensitivity/eps)
            dist += np.linalg.norm(released_mean - true_mean)
        dist /= num_trials
        avg_dist_dp[i] = (clip_dist, dist)
    dp_key = min(avg_dist_dp.items(), key=lambda x: x[1][1])[0]
    dp_dists[eps] = avg_dist_dp[dp_key]


In [74]:
dp_dists

{11.51291546492478: (0.0033028177886092872, 0.02264604587884551),
 1.6426117097961406: (0.029059525838498572, 0.1484504726368023),
 0.7304317044395013: (0.037669177813089136, 0.3253594708011076),
 0.3563228120191924: (0.11073965010428659, 0.640035299480314)}

In [75]:
# PAC MEAN
train_x, train_y, test_x, test_y, num_classes, train_len = gen_iris(normalize=False)
true_mean = np.average(train_x, axis=0)

norms = [np.linalg.norm(x) for x in train_x]
# print(max(norms))

subsample_rate = int(0.5*train_len)

noise = hybrid_noise_auto(train_x, train_y, subsample_rate, num_classes, 1e-6)

pac_dists = {}
num_trials = 1000

for mi in mi_range:
    scaled_noise = {k: noise[k] * (0.5 / mi) for k in noise}
    iso_noise = max(scaled_noise.values())
    iso_scaled = {k: iso_noise for k in noise}
    avg_dist_pac = 0
    avg_iso_dist_pac = 0
    subsampled_dist = 0
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        subsampled_dist += np.linalg.norm(released_mean - true_mean)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=scaled_noise[ind])
            released_mean[ind] += c
        avg_dist_pac += np.linalg.norm(released_mean - true_mean)
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=iso_scaled[ind])
            released_mean[ind] += c
        avg_iso_dist_pac += np.linalg.norm(released_mean - true_mean)
    avg_iso_dist_pac /= num_trials
    avg_dist_pac /= num_trials
    subsampled_dist /= num_trials
    pac_dists[mi] = (subsampled_dist, avg_dist_pac, avg_iso_dist_pac)

In [76]:
pac_dists

{1.0: (0.1833370805904014, 0.18488085550303998, 0.18672833086769447),
 0.25: (0.17155257051618994, 0.22048175977548357, 0.30099991543834453),
 0.0625: (0.17315374929370478, 0.5727056254379056, 0.933757243359157),
 0.015625: (0.17599550552376558, 2.1872905843053476, 3.733886299234875)}

In [77]:
train_x, train_y, test_x, test_y, num_classes, train_len = gen_bean(normalize=True)

In [78]:
true_mean = np.average(train_x, axis=0)

norms = [np.linalg.norm(x) for x in train_x]
print(max(norms))

2.989481085350122


In [79]:
true_mean

array([0.13983595, 0.22699883, 0.24666734, 0.23664163, 0.39805406,
       0.76834721, 0.13689441, 0.22559247, 0.62622439, 0.89969561,
       0.76509316, 0.45884769, 0.49295364, 0.3708321 , 0.41278467,
       0.90979207])

In [80]:
# DP MEAN
dp_dists = {}
num_trials = 1000

for eps in epsilon_vals:
    avg_dist_dp = {}
    for i in range(1, 31):
        clip_budget = i/10
        clipped_train_x = [clip_to_threshold(train_x[i], clip_budget) for i in range(len(train_x))]
        released_mean = np.average(clipped_train_x, axis=0)
        clip_dist = np.linalg.norm(released_mean - true_mean)
        dist = 0.
        for _ in range(num_trials):
            released_mean = np.average(clipped_train_x, axis=0)
            for ind in range(len(released_mean)):
                sensitivity = clip_budget / train_len 
                released_mean[ind] += add_noise(sensitivity / eps)
            dist += np.linalg.norm(released_mean - true_mean)
        dist /= num_trials
        avg_dist_dp[i] = (clip_dist, dist)
    dp_key = min(avg_dist_dp.items(), key=lambda x: x[1][1])[0]
    dp_dists[eps] = avg_dist_dp[dp_key]


In [81]:
dp_dists

{11.51291546492478: (0.0, 0.00015006824580085908),
 1.6426117097961406: (0.00022235407709886266, 0.0010111456603110356),
 0.7304317044395013: (0.00022235407709886266, 0.0021773410426364865),
 0.3563228120191924: (0.0008748404929961534, 0.004455310566051728)}

In [82]:
[dp_dists[x] for x in dp_dists]

[(0.0, 0.00015006824580085908),
 (0.00022235407709886266, 0.0010111456603110356),
 (0.00022235407709886266, 0.0021773410426364865),
 (0.0008748404929961534, 0.004455310566051728)]

In [83]:
# PAC MEAN
subsample_rate = int(0.5*train_len)

noise = hybrid_noise_auto(train_x, train_y, subsample_rate, num_classes, 1e-6)

pac_dists = {}
num_trials = 1000

for mi in mi_range:
    scaled_noise = {k: noise[k] * (0.5 / mi) for k in noise}
    iso_noise = max(scaled_noise.values())
    iso_scaled = {k: iso_noise for k in noise}
    avg_dist_pac = 0
    avg_iso_dist_pac = 0
    subsampled_dist = 0
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        subsampled_dist += np.linalg.norm(released_mean - true_mean)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=scaled_noise[ind])
            released_mean[ind] += c
        avg_dist_pac += np.linalg.norm(released_mean - true_mean)
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=iso_scaled[ind])
            released_mean[ind] += c
        avg_iso_dist_pac += np.linalg.norm(released_mean - true_mean)
    avg_iso_dist_pac /= num_trials
    avg_dist_pac /= num_trials
    subsampled_dist /= num_trials
    pac_dists[mi] = (subsampled_dist, avg_dist_pac, avg_iso_dist_pac)

In [84]:
pac_dists

{1.0: (0.005419380181007873, 0.005419772757532086, 0.005360990985781511),
 0.25: (0.005518133864130868, 0.00552266447098752, 0.005364708104027236),
 0.0625: (0.005517902070771221, 0.005597313450164743, 0.005460130609496094),
 0.015625: (0.005323597408348473, 0.006379864958697195, 0.007060470820374045)}

In [85]:
# RICE

In [86]:
train_x, train_y, test_x, test_y, num_classes, train_len = gen_rice(normalize=True)

In [87]:
true_mean = np.average(train_x, axis=0)

norms = [np.linalg.norm(x) for x in train_x]
print(max(norms))

2.399557199470374


In [88]:
true_mean

array([0.45343291, 0.50607844, 0.46734053, 0.55984097, 0.6425448 ,
       0.46285337, 0.45145814])

In [89]:
# DP MEAN
dp_dists = {}
num_trials = 1000

for eps in epsilon_vals:
    avg_dist_dp = {}
    for i in range(1, 3):
        clip_budget = i
        clipped_train_x = [clip_to_threshold(train_x[i], clip_budget) for i in range(len(train_x))]
        released_mean = np.average(clipped_train_x, axis=0)
        clip_dist = np.linalg.norm(released_mean - true_mean)
        dist = 0.
        for _ in range(num_trials):
            released_mean = np.average(clipped_train_x, axis=0)
            for ind in range(len(released_mean)):
                sensitivity = clip_budget / train_len 
                released_mean[ind] += add_noise(sensitivity / eps)
            dist += np.linalg.norm(released_mean - true_mean)
        dist /= num_trials
        avg_dist_dp[i] = (clip_dist, dist)
    dp_key = min(avg_dist_dp.items(), key=lambda x: x[1][1])[0]
    dp_dists[eps] = avg_dist_dp[dp_key]


In [90]:
dp_dists

{11.51291546492478: (0.0017379061286585567, 0.0017539046053067751),
 1.6426117097961406: (0.0017379061286585567, 0.002354901839435473),
 0.7304317044395013: (0.0017379061286585567, 0.0039036176494739692),
 0.3563228120191924: (0.0017379061286585567, 0.007494171497036016)}

In [91]:
# PAC MEAN
subsample_rate = int(0.5*train_len)

noise = hybrid_noise_auto(train_x, train_y, subsample_rate, num_classes, 1e-6)

pac_dists = {}
num_trials = 1000

for mi in mi_range:
    scaled_noise = {k: noise[k] * (0.5 / mi) for k in noise}
    iso_noise = max(scaled_noise.values())
    iso_scaled = {k: iso_noise for k in noise}
    avg_dist_pac = 0
    avg_iso_dist_pac = 0
    subsampled_dist = 0
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        subsampled_dist += np.linalg.norm(released_mean - true_mean)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=scaled_noise[ind])
            released_mean[ind] += c
        avg_dist_pac += np.linalg.norm(released_mean - true_mean)
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=iso_scaled[ind])
            released_mean[ind] += c
        avg_iso_dist_pac += np.linalg.norm(released_mean - true_mean)
    avg_iso_dist_pac /= num_trials
    avg_dist_pac /= num_trials
    subsampled_dist /= num_trials
    pac_dists[mi] = (subsampled_dist, avg_dist_pac, avg_iso_dist_pac)

In [92]:
pac_dists

{1.0: (0.007755244548979158, 0.007757153440237647, 0.00788784672455717),
 0.25: (0.007606838552760716, 0.007616898395780751, 0.007523157190096676),
 0.0625: (0.007673844887516511, 0.007769162105991995, 0.0077546563454638775),
 0.015625: (0.007929341416813323, 0.00968174069410722, 0.01093161455184949)}